# batch size 에 따른 inference 속도 비교

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import collections
slim = tf.contrib.slim

In [5]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [17]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

x_img = tf.reshape(X, [-1, 28, 28, 1])

# slim.conv2d uses relu and xavier init as default
h1 = slim.conv2d(x_img, 64, kernel_size=[3,3]) 
h1 = slim.conv2d(x_img, 64, kernel_size=[3,3]) 
h1 = slim.conv2d(x_img, 64, kernel_size=[3,3]) 
p1 = slim.max_pool2d(h1, kernel_size=[2,2])
# checking dimensions for each layer is very helpful
assert p1.shape[1:] == [14, 14, 64]

h2 = slim.conv2d(p1, 128, kernel_size=[3,3])
h2 = slim.conv2d(p1, 128, kernel_size=[3,3])
h2 = slim.conv2d(p1, 128, kernel_size=[3,3])
p2 = slim.max_pool2d(h2, kernel_size=[2,2])
assert p2.shape[1:] == [7, 7, 128]

flat = slim.flatten(p2)
logits = slim.fully_connected(flat, 10, activation_fn=None)
prob = tf.nn.softmax(logits)

correct = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(loss)
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [18]:
epoch_n=10
batch_size=100
n_iter=None

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

N = mnist.train.num_examples
if n_iter is None:
    n_iter = N // batch_size
# the last 5 accuracies will be used for the evaluation of a model
dq = collections.deque(maxlen=5)

for epoch in range(epoch_n):
    avg_loss = 0.
    avg_acc = 0.
    for _ in range(n_iter):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        _, cur_acc, cur_loss = sess.run([train_op, accuracy, loss], {X: batch_x, y: batch_y})
        avg_acc += cur_acc
        avg_loss += cur_loss

    avg_acc /= n_iter
    avg_loss /= n_iter

    test_acc = 0.
    test_loss = 0.
    for _ in range(mnist.test.num_examples // 1000):
        batch_x, batch_y = mnist.test.next_batch(1000)
        cur_acc, cur_loss = sess.run([accuracy, loss], {X: batch_x, y: batch_y})
        test_acc += cur_acc
        test_loss += cur_loss
    test_acc /= (mnist.test.num_examples // 1000)
    test_loss /= (mnist.test.num_examples // 1000)

    print("[{:2}/{}] (train) acc: {:.2%}, loss: {:.3f} | (test) acc: {:.2%}, loss: {:.3f}".
          format(epoch+1, epoch_n, avg_acc, avg_loss, test_acc, test_loss))
    dq.append(test_acc)

score = np.average(dq)
print("average of last 5 test acc: {:.2%}".format(score))

[ 1/10] (train) acc: 94.20%, loss: 0.194 | (test) acc: 98.24%, loss: 0.055
[ 2/10] (train) acc: 98.36%, loss: 0.055 | (test) acc: 98.85%, loss: 0.037
[ 3/10] (train) acc: 98.73%, loss: 0.039 | (test) acc: 98.86%, loss: 0.034
[ 4/10] (train) acc: 99.03%, loss: 0.031 | (test) acc: 99.03%, loss: 0.030
[ 5/10] (train) acc: 99.23%, loss: 0.023 | (test) acc: 98.88%, loss: 0.037
[ 6/10] (train) acc: 99.40%, loss: 0.019 | (test) acc: 98.83%, loss: 0.035
[ 7/10] (train) acc: 99.51%, loss: 0.016 | (test) acc: 99.11%, loss: 0.029
[ 8/10] (train) acc: 99.57%, loss: 0.012 | (test) acc: 99.11%, loss: 0.029
[ 9/10] (train) acc: 99.60%, loss: 0.012 | (test) acc: 98.77%, loss: 0.043
[10/10] (train) acc: 99.72%, loss: 0.009 | (test) acc: 99.05%, loss: 0.031
average of last 5 test acc: 98.97%


In [22]:
%%timeit
pr = sess.run(prob, {X: mnist.test.images[:8]})

1000 loops, best of 3: 845 µs per loop


In [23]:
%%timeit
pr = sess.run(prob, {X: mnist.test.images[:1]})

1000 loops, best of 3: 563 µs per loop


In [24]:
import time

In [37]:
n_loops = 1000

for k in range(10):
    size = 2**k
#     print size
    
    st_time = time.time()
    
    for i in range(n_loops):
        st = np.random.randint(low=0, high=10000-size)
        pr = sess.run(prob, {X: mnist.test.images[st:st+size]})
        
    print("batch_size: {} / time: {}".format(size, time.time() - st_time))

batch_size: 1 / time: 0.779050111771
batch_size: 2 / time: 0.716285943985
batch_size: 4 / time: 0.821782827377
batch_size: 8 / time: 0.76958990097
batch_size: 16 / time: 0.850898981094
batch_size: 32 / time: 1.59397101402
batch_size: 64 / time: 2.02984499931
batch_size: 128 / time: 2.69542598724
batch_size: 256 / time: 3.91907596588
batch_size: 512 / time: 6.30875396729


6040